# 16 - Reindeer Maze

https://adventofcode.com/2024/day/16


In [1]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";
import images from "../../utils/images.ts";
import arrays from "../../utils/arrays.ts";

In [2]:
// Read Input

const file = await Deno.readTextFile("input-base.txt");
const area = file.split("\n").map((line) => line.split(""));
arrays.print2D(area)

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############


In [3]:
// Prepare Data

const stripBoundary = (area: string[][]) => {
    return area.filter((row, i) => i !== 0 && i !== area.length - 1).map((row) => row.filter((cell, i) => i !== 0 && i !== row.length - 1));
}

const reversedStrippedArea = stripBoundary(objects.deepCopy(area)).reverse();
arrays.print2D(reversedStrippedArea)

S..#.....#...
.###.#.#.#.#.
.....#...#.#.
.#.#.###.#.#.
...#.....#.#.
##.#.#####.#.
...........#.
.#.#####.###.
.#.#.......#.
.###.#####.#.
.....#.#...#.
.#.###.#.###.
.......#....E


In [4]:
// Part 1 - What is the lowest score a Reindeer could possibly get?

const isWall = el => el === '#';

const nextRightAngleDirectionIDs = (directionID: string) => {
    switch (directionID) {
        case '^':
            return ['<', '>'];
        case 'v':
            return ['>', '<'];
        case '<':
            return ['v', '^'];
        case '>':
            return ['^', 'v'];
    }
}

const getDirection = (directionID: string) => {
    switch (directionID) {
        case '^':
            return [-1, 0];
        case 'v':
            return [1, 0];
        case '<':
            return [0, -1];
        case '>':
            return [0, 1];
    }
}

const pathKey = (i, j, direction) => `${i}-${j}-${direction}`;
const visitedKey = (i, j) => `${i}-${j}`;


const populateMinMovesCost = (area: string[][], startPosition: [], startDirectionID: string) => {
    const stack = [{ position: startPosition, directionID: startDirectionID, path: new Set<string>(), cost: 0, visited: new Set() }];

    let minCost = Number.MAX_SAFE_INTEGER;
    let minPaths = [];
    const globallyVisitedCost = {}

    while (stack.length > 0) {
        const { position, directionID, path, cost, visited } = stack.shift();
        const [i, j] = position;

        if (i < 0 || i >= area.length || j < 0 || j >= area[0].length) {
            continue;
        }

        const currentCell = area[i][j];
        const currentPathKey = pathKey(i, j, directionID);

        const previousMinCostOnCurrentCell = globallyVisitedCost[currentPathKey];
        if (!previousMinCostOnCurrentCell) globallyVisitedCost[currentPathKey] = cost;
        if (previousMinCostOnCurrentCell) {
            if (previousMinCostOnCurrentCell < cost) {
                continue;
            }
            else {
                globallyVisitedCost[currentPathKey] = cost;
            }
        }

        path.add(currentPathKey);
        if (currentCell === 'E' && minCost >= cost) {
            if (minCost > cost) minPaths = []
            minPaths.push(path);
            minCost = cost;
            continue;
        }

        if (cost > minCost) {
            continue;
        }

        const direction = getDirection(directionID);
        const nextInCurrentDirection = area[i + direction[0]] ? area[i + direction[0]][j + direction[1]] : null;
        if (nextInCurrentDirection && !isWall(nextInCurrentDirection)) {
            stack.push({ position: [i + direction[0], j + direction[1]], directionID, path: new Set([...path]), cost: cost + 1, visited: new Set([...visited]) });
        }

        nextRightAngleDirectionIDs(directionID).forEach((d) => {
            const nextInNewDirection = area[i + getDirection(d)[0]] ? area[i + getDirection(d)[0]][j + getDirection(d)[1]] : null;
            if (nextInNewDirection && !isWall(nextInNewDirection)) {
                stack.push({ position: [i + getDirection(d)[0], j + getDirection(d)[1]], directionID: d, path: new Set([...path]), cost: cost + 1001, visited: new Set([...visited]) });
            }
        });
    }
    return [minCost, minPaths];
}

const currentArea = objects.deepCopy(reversedStrippedArea);
const [minCost, minPaths] = populateMinMovesCost(currentArea, [0, 0], '>', new Set(), 0);
[...minPaths[0]].forEach((pathKey, index) => {
    if (index === 0) return;
    const [i, j, direction] = pathKey.split('-');
    currentArea[i][j] = direction;
});
colors.visualize2DArray(currentArea, {
    colors: {
        '^': colors.bgRed,
        'v': colors.bgRed,
        '<': colors.bgRed,
        '>': colors.bgRed,
        'E': colors.bgGreen,
        'S': colors.bgGreen,
        '#': colors.bgBlack,
    }
});

minCost

7036

In [5]:
// Part - 2 How many tiles are part of at least one of the best paths through the maze?

const uniqueTilesOnMinPaths = new Set();
minPaths.forEach((path) => {
    [...path].forEach((pathKey) => {
        const [i, j, _] = pathKey.split('-');
        uniqueTilesOnMinPaths.add(`${i}-${j}`);
    });
});
// console.log(uniqueTilesOnMinPaths);
uniqueTilesOnMinPaths.size

45